<a href="https://colab.research.google.com/github/xpeng-26/Erdos_Finance_LLM_Project/blob/Jerry_project/Notebooks/train_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!git clone --single-branch --branch Jerry_project https://github.com/xpeng-26/Erdos_Finance_LLM_Project.git

Cloning into 'Erdos_Finance_LLM_Project'...
remote: Enumerating objects: 544, done.
remote: Counting objects: 100% (212/212), done.
remote: Compressing objects: 100% (102/102), done.
remote: Total 544 (delta 131), reused 168 (delta 98), pack-reused 332 (from 1)
Receiving objects: 100% (544/544), 9.83 MiB | 38.00 MiB/s, done.
Resolving deltas: 100% (285/285), done.


In [3]:
!pip install tomli_w
!pip install torchinfo

In [4]:
%cd Erdos_Finance_LLM_Project/src

/content/Erdos_Finance_LLM_Project/src


In [5]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Tue Apr 22 05:18:10 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   38C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
import os
import sys
from datetime import datetime
import warnings

# Add ../src to the path
proj_path = os.path.abspath(os.path.join(os.getcwd(), "../src"))
sys.path.insert(0, proj_path)

# Import the parse config function to parse the .toml file
from utils.config_tool import parse_config
from utils.logging_tool import initialize_logger
from strategy.train_trading_agent import train_trading_agent

# Get the time
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

# Load the config file
config = parse_config("../config/trade_w_llm.toml")
# Initialize the logger
log_file = "{}_log_{}.txt".format(
			'training_agent',
			str(timestamp))
logger = initialize_logger(log_path=os.path.join(config['info']['local_data_path'], 'logs'),log_file=log_file)

with warnings.catch_warnings():
					warnings.filterwarnings("ignore", category=UserWarning)
					train_trading_agent(config, logger)

INFO:utils.logging_tool:Data loaded from /content/drive/MyDrive/Erdos_2025_spring/dataset/data_raw/stock_daily_data.db, extract ticker: ['AAPL', 'MSFT', 'NVDA', 'GOOG']
INFO:utils.logging_tool:Environment: portfolio
INFO:utils.logging_tool:==========================================================================================
Layer (type:depth-idx)                   Output Shape              Param #
MultiHeadNetwork                         [1, 11]                   --
├─ModuleList: 1-1                        --                        --
│    └─Linear: 2-1                       [1, 256]                  37,120
│    └─ReLU: 2-2                         [1, 256]                  --
│    └─Linear: 2-3                       [1, 128]                  32,896
│    └─ReLU: 2-4                         [1, 128]                  --
│    └─Linear: 2-5                       [1, 64]                   8,256
│    └─ReLU: 2-6                         [1, 64]                   --
├─Dropout: 1-2         